In [1]:
!pip install git+https://github.com/Seokhyeon-Park/hanspell.git

  Cloning https://github.com/Seokhyeon-Park/hanspell.git to /tmp/pip-req-build-ol9r6_qd
  Running command git clone --filter=blob:none --quiet https://github.com/Seokhyeon-Park/hanspell.git /tmp/pip-req-build-ol9r6_qd
  Resolved https://github.com/Seokhyeon-Park/hanspell.git to commit 87c740ec6ad6f8c418fcac653d0e5d8ae5b7c1d9
  Preparing metadata (setup.py) ... done
  Created wheel for hanspell: filename=hanspell-0.1-py3-none-any.whl size=3895 sha256=0834c8870e2e67aea8637a6a8d9739b528906ac66d8483f3685e696a56510efc
  Stored in directory: /tmp/pip-ephem-wheel-cache-u_uqp90v/wheels/97/15/22/6b178ab85fe53e90b1f93966e35b49a2c5321ae3b58c0e4019
Successfully built hanspell


In [2]:
import hanspell
print(hanspell.__file__)

/usr/local/lib/python3.10/dist-packages/hanspell/__init__.py


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import zipfile
import os
from os import ttyname
from hanspell import spell_checker
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

In [22]:
# zipfile 압축 풀기
def unzip_file(base_path, zip_file_name):
    zip_file_path = os.path.join(base_path, zip_file_name)
    extract_file_name = zip_file_name[:-4] #.zip 제거해서 폴더이름 생성
    extract_path = os.path.join(base_path, extract_file_name)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(zip_file_path[:-4])

    file_names = os.listdir(extract_path)
    print(f"successfully unzipped and extracted files from {zip_file_name}")

    return extract_file_name, file_names

In [23]:
# 여러개의 zipfile 한번에 압축풀기
def unzip_files(base_path, zip_file_names):
    unzip_file_names = {}

    for zip_file_name in zip_file_names:
        extract_file_name, unzip_file_name = unzip_file(base_path, zip_file_name)
        unzip_file_names[extract_file_name] = unzip_file_name

    return unzip_file_names

In [29]:
def json_to_df(folder_name, file_names, base_path):
    speaker_type_list = []
    text_list = []

    for file_name in file_names:
        if file_name.endswith(".json"):
            folder_path = os.path.join(base_path, folder_name)
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r', encoding = "utf-8") as file:
                data = json.load(file)
                info = data.get("info")[0]
                annot = info.get("annotations")

                speaker_type = annot.get("speaker_type")
                text = annot.get("text")

                speaker_type_list.append(speaker_type)
                text_list.append(text)

    data_dict = {"speaker_type": speaker_type_list,
                 "conversation": text_list}

    df = pd.DataFrame(data_dict)
    print("successfully transformed json to DataFrame: ", folder_name)
    df.head()

    return df

In [30]:
# json -> pd DataFrame 변환
def json_to_dfs(unzip_file_names, base_path):
    dfs = []

    for folder_name, file_names in unzip_file_names.items():
        df = json_to_df(folder_name, file_names, base_path)
        dfs.append(df)

    final_df = pd.concat(dfs, axis = 0, ignore_index = True)
    return final_df

In [32]:
# 1차 텍스트 전처리
def clean_text(text):
    cleaned_text = re.sub(r'\d\s?:|~', ' ', text)
    cleaned_text = re.sub(r'[“”"]', '', cleaned_text)
    cleaned_text = cleaned_text.strip()
    return cleaned_text

In [33]:
# df 변수 정리하기 및 1차 텍스트 전처리
def clean_df(df):
    df = df[df['speaker_type'] == "1:1"]
    df['class'] = '일반 대화'
    df.reset_index(inplace = True)
    df = df[['index', 'class', 'conversation']]
    df.rename({"index": "idx"}, axis = 1, inplace = True)
    df['conversation'] = df['conversation'].apply(clean_text)

    return df

In [4]:
base_path = "/content/drive/MyDrive/Colab Notebooks/[아이펠] 리서치과정 10기/DLThon/data/AIHub/020.주제별 텍스트 일상 대화 데이터/01.데이터/1.Training/라벨링데이터"
zip_file_names = os.listdir(base_path)

In [8]:
zip_file_names = zip_file_names[1:4]
zip_file_names

['TL_05. NATEON.zip', 'TL_02. FACEBOOK.zip', 'TL_04. BAND.zip']

In [24]:
unzip_file_names = unzip_files(base_path, zip_file_names)
unzip_file_names

successfully unzipped and extracted files from TL_05. NATEON.zip
successfully unzipped and extracted files from TL_02. FACEBOOK.zip
successfully unzipped and extracted files from TL_04. BAND.zip


{'TL_05. NATEON': ['NATEON_21_05.json',
  'NATEON_21_06.json',
  'NATEON_21_07.json',
  'NATEON_21_08.json',
  'NATEON_21_09.json',
  'NATEON_21_10.json',
  'NATEON_21_11.json',
  'NATEON_21_12.json',
  'NATEON_21_13.json',
  'NATEON_21_14.json',
  'NATEON_21_15.json',
  'NATEON_21_16.json',
  'NATEON_21_17.json',
  'NATEON_21_18.json',
  'NATEON_21_19.json',
  'NATEON_21_20.json',
  'NATEON_22_01.json',
  'NATEON_22_02.json',
  'NATEON_22_03.json',
  'NATEON_22_04.json',
  'NATEON_22_05.json',
  'NATEON_22_06.json',
  'NATEON_22_07.json',
  'NATEON_22_08.json',
  'NATEON_22_09.json',
  'NATEON_22_10.json',
  'NATEON_22_11.json',
  'NATEON_22_12.json',
  'NATEON_22_13.json',
  'NATEON_22_14.json',
  'NATEON_22_15.json',
  'NATEON_22_16.json',
  'NATEON_22_17.json',
  'NATEON_22_18.json',
  'NATEON_22_19.json',
  'NATEON_22_20.json',
  'NATEON_23_01.json',
  'NATEON_23_02.json',
  'NATEON_23_03.json',
  'NATEON_23_04.json',
  'NATEON_23_05.json',
  'NATEON_23_06.json',
  'NATEON_23_07.j

In [31]:
df = json_to_dfs(unzip_file_names, base_path)
df.head()

successfully transformed json to DataFrame:  TL_05. NATEON
successfully transformed json to DataFrame:  TL_02. FACEBOOK
successfully transformed json to DataFrame:  TL_04. BAND


,speaker_type,conversation
0,1:1,1 : 군대에 다녀오셨나요?\n2 : 네 군대에 다녀왔습니다\n2 : 군대에 다녀오...
1,1:1,1 : 학원 다니신 적 있으세요?\n2 : 중학교 때 다녔어요\n2 : 학원 다니신...
2,1:1,1 : 부모님과 관계가 좋나요?\n2 : 네 괜찮은 거 같아요\n2 : 부모님과 관...
3,1:1,1 : 연애를 하고 계시나요?\n2 : 네 하고 있어요\n2 : 연애를 하고 계신가...
4,1:1,1 : 요즘 코로나 시대로 인해서 반려 동물을 많이 찾는다고 하는데 키우고 싶은 반...


In [35]:
df = clean_df(df)
df.head()

<ipython-input-33-47eba32e1272>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'] = '일반 대화'


,idx,class,conversation
0,0,일반 대화,군대에 다녀오셨나요?\n 네 군대에 다녀왔습니다\n 군대에 다녀오셨나요?\n ...
1,1,일반 대화,학원 다니신 적 있으세요?\n 중학교 때 다녔어요\n 학원 다니신 적 있으신가요...
2,2,일반 대화,부모님과 관계가 좋나요?\n 네 괜찮은 거 같아요\n 부모님과 관계가 좋으신가요...
3,3,일반 대화,연애를 하고 계시나요?\n 네 하고 있어요\n 연애를 하고 계신가요?\n 네 ...
4,4,일반 대화,요즘 코로나 시대로 인해서 반려 동물을 많이 찾는다고 하는데 키우고 싶은 반려 동물...


In [ ]:
df.to_csv(os.path.join(base_path, "TL_020405_save.csv"), index = False)

# 2. 데이터 정규화 및 Hanspell 적용

In [6]:
# 데이터 로드 함수 정의
def load_data(data_path):
    data = pd.read_csv(data_path)
    print("총 데이터 수: ", len(data))
    print("결측치 확인: \n", data.isnull().sum())
    print("중복 데이터 확인: ", sum(data.duplicated()))

    data = data.dropna()
    data = data.drop_duplicates()
    print("결측치 및 중복 제거 후 총 데이터 수: ", len(data))
    print(data.head())
    return data

In [7]:
# 텍스트 정규화
def normalize_sentence(sentence):
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9?.!,]+", " ", sentence) # 특수문자 제거
    sentence = re.sub(r"\n", ' ', sentence) # 줄바꿈 제거
    sentence = re.sub(r"\s+", " ", sentence) # 공백 제거
    sentence = re.sub(r"\?+", "?", sentence) # 여러개의 연속된 물음표 하나로 변환
    sentence = sentence.strip()
    return sentence

In [8]:
# 한글 맞춤법 검사
def apply_hanspell(idx, row):
    try:
        result = spell_checker.check(row).as_dict().get("checked")
        if len(result) > 1:
            return result
        else:
            print("length of the row is below 1: ", idx)
            return row
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        return row

In [9]:
def preprocess_data(data):
    data['conversation'] = data['conversation'].apply(normalize_sentence)
    data['conversation'] = [apply_hanspell(idx, row) for idx, row in tqdm(enumerate(data['conversation']), total = len(data))]
    return data

In [10]:
def check_length(data):
    print("total count of indices: ", len(data))
    data["text_len"] = data["conversation"].apply(lambda x: len(x))
    print(f"min length: {np.min(data['text_len'])}")
    print(f"max length: {np.max(data['text_len'])}")
    print(f"avg length: {np.average(data['text_len'])}")
    print(f"num of indices which length is below 3: {len(data[data['text_len'] < 3])}")
    data.drop(["text_len"], axis =1, inplace = True)

In [14]:
def count_long_conversations(df, column_name='conversation', length_threshold=500):
    """
    DataFrame에서 지정한 열(column_name)의 값 중 길이가 length_threshold 이상인 값의 개수를 반환하는 함수.

    Parameters:
    - df (pd.DataFrame): DataFrame 객체
    - column_name (str): 길이를 계산할 열 이름 (기본값: 'conversation')
    - length_threshold (int): 길이 기준 값 (기본값: 500)

    Returns:
    - int: 길이가 기준값 이상인 데이터의 개수
    """
    # conversation 열의 길이를 계산하여 필터링
    long_conversations = df[df[column_name].apply(lambda x: len(x) if isinstance(x, str) else 0) >= length_threshold]
    return len(long_conversations)


In [12]:
data_path = os.path.join(base_path, "TL_020405_save.csv")
data = load_data(data_path)

총 데이터 수:  5783
결측치 확인: 
 idx             0
class           0
conversation    0
dtype: int64
중복 데이터 확인:  0
결측치 및 중복 제거 후 총 데이터 수:  5783
   idx  class                                       conversation
0    0  일반 대화  군대에 다녀오셨나요?\n  네 군대에 다녀왔습니다\n  군대에 다녀오셨나요?\n  ...
1    1  일반 대화  학원 다니신 적 있으세요?\n  중학교 때 다녔어요\n  학원 다니신 적 있으신가요...
2    2  일반 대화  부모님과 관계가 좋나요?\n  네 괜찮은 거 같아요\n  부모님과 관계가 좋으신가요...
3    3  일반 대화  연애를 하고 계시나요?\n  네 하고 있어요\n  연애를 하고 계신가요?\n  네 ...
4    4  일반 대화  요즘 코로나 시대로 인해서 반려 동물을 많이 찾는다고 하는데 키우고 싶은 반려 동물...


In [15]:
count_long_conversations(data)

255

In [ ]:
data = preprocess_data(data)

  0%|          | 0/5783 [00:00<?, ?it/s]

length of the row is below 1:  0
length of the row is below 1:  1
length of the row is below 1:  2
length of the row is below 1:  3
length of the row is below 1:  4
length of the row is below 1:  5
length of the row is below 1:  6
length of the row is below 1:  7
length of the row is below 1:  8
length of the row is below 1:  9
length of the row is below 1:  10
Error at index 15: Expecting value: line 1 column 1 (char 0)
length of the row is below 1:  16
length of the row is below 1:  17
length of the row is below 1:  18
length of the row is below 1:  19
length of the row is below 1:  20
length of the row is below 1:  21
length of the row is below 1:  22
length of the row is below 1:  23
length of the row is below 1:  24
length of the row is below 1:  25
length of the row is below 1:  26
length of the row is below 1:  27
length of the row is below 1:  28
length of the row is below 1:  29
length of the row is below 1:  30
length of the row is below 1:  31
length of the row is below 1:  

In [ ]:
check_length(data)

In [ ]:
save_path = "/content/drive/MyDrive/Colab Notebooks/[아이펠] 리서치과정 10기/DLThon/data/AIHub/final_dataset"
file_name = "train_dataset_to_add_fb2band_241208.csv"
data.to_csv(os.path.join(save_path, file_name), index = False)